In [1]:
%matplotlib inline
import numpy as np
import pylab as pl
import scipy.special as special
from scipy.integrate import quad

# Set plot parameters to make beautiful plots
pl.rcParams['figure.figsize']  = 12, 7.5
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 20  
pl.rcParams['font.sans-serif'] = 'serif'
pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'large'

pl.rcParams['xtick.major.size'] = 8     
pl.rcParams['xtick.minor.size'] = 4     
pl.rcParams['xtick.major.pad']  = 8     
pl.rcParams['xtick.minor.pad']  = 8     
pl.rcParams['xtick.color']      = 'k'     
pl.rcParams['xtick.labelsize']  = 'large'
pl.rcParams['xtick.direction']  = 'in'    

pl.rcParams['ytick.major.size'] = 8     
pl.rcParams['ytick.minor.size'] = 4     
pl.rcParams['ytick.major.pad']  = 8     
pl.rcParams['ytick.minor.pad']  = 8     
pl.rcParams['ytick.color']      = 'k'     
pl.rcParams['ytick.labelsize']  = 'large'
pl.rcParams['ytick.direction']  = 'in'

In [2]:
# EVERYTHING IS 1 units; NOT SURE IF THESE ARE INTERNALLY CONSISTENT

#constants
e        = 1.     #electron charge
m        = 1.     #electron mass
c        = 1.     #speed of light
epsilon0 = 1.     #permittivity of free space
epsilon  = -1.    #sign of electron charge

#parameters
B     = 1.         #background B strength
n_e   = 1.         #electron number density cm^-3
w_T   = 1.        #dimensionless electron temp. k_B T / m c^2
#w_T   = 134071263.046
theta = np.pi / 3. #observer angle

#derived quantities
omega_p = np.sqrt(n_e * e**2. / (m * epsilon0))     # plasma frequency    (=1 in these units)
omega_c = e * B / (m * c)                           # cyclotron frequency (=1 in these units)

In [88]:
#OUR FORMULATION OF THE NR MAXWELLIAN K_12

def K_12_integral(tau, omega):
    k_perp = omega / c * np.sin(theta)                  # wavevector perp component n = 1 approximation
    k_z    = omega / c * np.cos(theta)                  # wavevector parallel comp. n = 1 approximation
    lambd  = k_perp**2. * w_T**2. / (2. * omega_c**2.)  # just a defined parameter
    term1 = -2. * 1j * omega_p**2. / omega
    term2 = np.sin(epsilon * omega_c * tau) * np.exp(1j * omega * tau)
    term3 = (lambd * np.sin(epsilon * omega_c * tau / 2.)**2. - 1./2.)
    term4 = np.exp(-2. * lambd * np.sin(epsilon * omega_c * tau / 2.)**2.)
    term5 = np.exp(-w_T**2. * k_z**2. * tau**2. / 4.)
    ans = term1 * term2 * term3 * term4 * term5
    return ans

#uses a nondimensionalized version of the integrand
def K_12_integral_scaled(tau, omega):
    k_perp = omega / c * np.sin(theta)                  # wavevector perp component n = 1 approximation
    k_z    = omega / c * np.cos(theta)                  # wavevector parallel comp. n = 1 approximation
    lambd  = k_perp**2. * w_T**2. / (2. * omega_c**2.)  # just a defined parameter
    term1 = -2. * 1j * omega_p**2. / omega / omega_c     #NOTE: before was -1 * this, but corrected error in deriv.
    term2 = np.sin(epsilon * 1. * tau) * np.exp(1j * omega * tau / omega_c)
    term3 = (lambd * np.sin(epsilon * 1. * tau / 2.)**2. - 1./2.)
    term4 = np.exp(-2. * lambd * np.sin(epsilon * 1. * tau / 2.)**2.)
    term5 = np.exp(-w_T**2. * k_z**2. * tau**2. / 4. / omega_c**2.)
    ans = term1 * term2 * term3 * term4 * term5
    return ans

def K_12_ours_scaled(omega):
    real_part = quad(lambda tau: K_12_integral_scaled(tau, omega).real, 0., np.inf)
    imag_part = quad(lambda tau: K_12_integral_scaled(tau, omega).imag, 0., np.inf)
    ans = real_part[0] + imag_part[0] * 1j
    return ans

def K_12_ours(omega):
    real_part = quad(lambda tau: K_12_integral(tau, omega).real, 0., np.inf)
    imag_part = quad(lambda tau: K_12_integral(tau, omega).imag, 0., np.inf)
    ans = real_part[0] + imag_part[0] * 1j
    return ans

omega = np.linspace(0.51, 4.51, 100)

def CPL(omega): #COLD PLASMA LIMIT
    ans = 1j * omega_c * omega_p**2. / (omega * (omega_c**2. - omega**2.))
    return ans

print K_12_ours(30.)
print K_12_ours_scaled(30.)

(3.01925993074e-05-2.42240556651e-05j)
(3.01925993074e-05-2.42240556651e-05j)


In [90]:
#SWANSON'S VERSION OF THE NR MAXWELLIAN K_12

#we first need the plasma dispersion function
def Z_integrand(xi, zeta):
    prefactor   = 1. / np.sqrt(np.pi)
    numerator   = np.exp(-xi**2.)
    
#    denominator = xi - zeta  #included in quad with weight type 'cauchy' passed to quad
    denominator = 1.
    
    return prefactor * numerator / denominator


#seems to work up to |zeta| = 625 where it's approx. -/+ 0.002 (so negative zeta yields +0.002)
def Z(zeta): 
    if(np.abs(zeta) < 2.):
        int_limit = 10. * np.abs(zeta)
    elif(np.abs(zeta) > 2. and np.abs(zeta) < 130.):
        int_limit = 2. * np.abs(zeta)
    else:
        int_limit = 1.5 * np.abs(zeta)
    
    imag_part = 1j * np.pi * Z_integrand(zeta, zeta)
    
    if(zeta != 0):
        ans = quad(lambda xi: Z_integrand(xi, zeta), -int_limit, int_limit, weight='cauchy', wvar=zeta)[0]
    else:
        ans = 0.
        
    return ans + imag_part

def K_12_summand(n, omega):
    k_perp = omega / c * np.sin(theta)                  # wavevector perp component n = 1 approximation
    k_z    = omega / c * np.cos(theta)                  # wavevector parallel comp. n = 1 approximation
    lambd   = k_perp**2. * w_T**2. / (2. * omega_c**2.) # just a defined parameter
    prefactor = 1j * epsilon * omega_p**2. * np.exp(-lambd) / (omega * k_z * w_T)
    zeta = (omega + n * omega_c) / (k_z * w_T)
    
    if(np.abs(zeta) > 625):
        print 'zeta out of range of PDF'
    
    term1 = n * (special.iv(n, lambd) - special.ivp(n, lambd)) * Z(zeta)
    ans = prefactor * term1
    return ans

def K_12_swanson(terms, omega):
    ans = 0.
    for i in range(-terms, terms):
        ans += K_12_summand(i, omega)
    return ans


#print K_12(number of sum terms evaluated, omega)
print K_12_swanson(100, 30.0)

(3.0192599307395053e-05-2.4227177539464392e-05j)


In [87]:
#seems to work
def AP_iv(n, x):
    ans = 1j**(-alpha) * special.jv(alpha, 1j*x)
    return np.real(ans)

#seems to work for integer n
def AP_ivp(n, x):   
    ans = 1j**(alpha + 1.) * special.jvp(alpha, 1j*x)
    
    #this factor probably comes from the derivative of the Bessel function
    #I didn't check, please check for me
    if(n % 2 != 0):
        ans = ans * -1.
    
    return np.real(ans)

alpha = 0.
x = 21.
print 'scipy:', special.iv(alpha, x), 'AP:', AP_iv(alpha, x), 'error:', (special.iv(alpha, x) - AP_iv(alpha, x))/special.iv(alpha, x)

print 'scipy:', special.ivp(alpha, x), 'AP', AP_ivp(alpha, x), 'error:', (special.ivp(alpha, x) - AP_ivp(alpha, x))/special.ivp(alpha, x)

scipy: 115513961.922158 AP: 115513961.92215806 error: -5.159951557678951e-16
scipy: 112729199.13777554 AP 112729199.13777553 error: 1.3218546133407489e-16
